In [1]:
import ibm_boto3
from ibm_botocore.client import Config

COS_ENDPOINT = "https://s3.eu-gb.cloud-object-storage.appdomain.cloud" # Bucket => Bucket_name => configuration => endpoints => public (put https:// in front of )
COS_API_KEY_ID = "your-api-key" # service credentilas => apikey
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token" # 
COS_RESOURCE_CRN = "your_resource_instace_id" # service credentials => resource_instace_id

cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),  
    endpoint_url=COS_ENDPOINT    
)

In [ ]:
!pip install music21

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [ ]:
import numpy as np
from music21 import converter, instrument, note, chord, midi, stream
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Dropout, Input, LSTM, BatchNormalization, Conv1D, TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
midi_files = []

for number in range(len(cos.list_objects(Bucket="music-test")["Contents"])):
    if ".mid" in cos.list_objects(Bucket="music-test")["Contents"][number]["Key"]:
        midi_files.append(cos.list_objects(Bucket="music-test")["Contents"][number]["Key"])

In [ ]:
for e in midi_files:
    cos.download_file(Bucket="music-test", Key=e, Filename=e)

In [ ]:
notes = []

for element in midi_files:
    midi = converter.parse(element)  
    notes_stream = midi.flat.notes
    for item in notes_stream: 
        if isinstance(item, note.Note): 
            notes.append(str(item.pitch))
        elif isinstance(item, chord.Chord):
            notes.append('.'.join(str(n) for n in item.normalOrder))

In [ ]:
node_number = 128
sequence_length = 100
notes_set = set(sorted(notes))
notes_set_length = len(set(sorted(notes)))
note_dict = {}

training_number = len(notes) - sequence_length
X = np.zeros((training_number, sequence_length, notes_set_length))
Y = np.zeros((training_number, notes_set_length))

for i, element in enumerate(notes_set):
    note_dict[element] = i
    
for i in range(0, training_number):
    input_sequence = notes[i: i+sequence_length]
    output_note = notes[i+sequence_length]
    for j, char in enumerate(input_sequence):
        X[i][j][note_dict[char]] = 1
    Y[i][note_dict[output_note]] = 1

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, strides=1, input_shape=(sequence_length, notes_set_length), activation = 'relu'))
#model.add(Dropout(0.1))
model.add(LSTM(node_number, return_sequences=True, input_shape=(sequence_length, notes_set_length), activation = 'sigmoid'))
#model.add(Dropout(0.1))
model.add(LSTM(node_number, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(notes_set_length))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
history = model.fit(X, Y, batch_size=128, epochs=1)
model.save('model_beeth_40.h5')

In [ ]:
cos.download_file('mozart','model_mz_50.h5','model_mz_50.h5')

In [ ]:
# load model
model = load_model('model_beeth_100.h5')

In [ ]:
note_dict_back = {}
for el in note_dict.keys():
    index = note_dict[el]
    note_dict_back[index] = el
    
n = np.random.randint(0, len(X)-1)
sequence = X[n]
start_sequence = sequence.reshape(1, sequence_length, notes_set_length)
output = []

for i in range(0, 100):
    new_note = model.predict(start_sequence, verbose=0)
    index = np.argmax(new_note)
    encoded_note = np.zeros((notes_set_length))
    encoded_note[index] = 1
    output.append(encoded_note)
    sequence = start_sequence[0][1:]
    start_sequence = np.concatenate((sequence, encoded_note.reshape(1, notes_set_length)))
    start_sequence = start_sequence.reshape(1, sequence_length, notes_set_length)
    
#for e in output:
#    print(e)

In [ ]:
final_notes = [] 
for elem in output:
    index = list(elem).index(1)
    final_notes.append(note_dict_back[index])
    
offset = 0
output_notes = []
    
for pattern in final_notes:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='output_beeth_40.mid')

In [ ]:
cos.upload_file(Bucket="beethoven1", Key="output_beeth_40.mid", Filename="output_beeth_40.mid")

In [ ]:
# upload model to cloud object storage
#cos.upload_file(Bucket="beethoven1", Key="model_beeth_40.h5", Filename="model_beeth_40.h5")